# What is Object-Oriented Programming?

Programming oriented around classes and objects

**"object"** : A structure with both data *and* functions

Similar to a C or IDL `struct`, but with functions "bound" to them.

# How do you use an object?

In [1]:
fobj = open('../README.md')
fobj.name

'../README.md'

In [2]:
fobj.readline()

'PyLunch Nova <sup>*</sup>\n'

# Some More Terminology

* **"method"**: A function that's attached to an object

* **"class"**: The "template" for an object (defines the methods and sometimes default data)

* **"instance"**: One particular object of a class (E.g. "that object is an instance of class File")

* **"instantiate/initialize"**: Creating a new instance of a class

To actually show this as a presentation, the easiest thing is to do ``jupyter nbconvert --to slides <this file's name>.ipynb --post serve``